<a href="https://colab.research.google.com/github/lennart194/thesis-code/blob/main/pooling_gwn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install ddop

In [2]:
import pandas as pd
import numpy as np

from ddop.newsvendor import GaussianWeightedNewsvendor
from ddop.newsvendor import SampleAverageApproximationNewsvendor

from sklearn.model_selection import RandomizedSearchCV

from ddop.metrics import make_scorer
from ddop.metrics import average_costs
from ddop.metrics import prescriptiveness_score

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## loading data-set


---



*   load the whole ultimative set
*   sort the rows first by item, then by date and set a multi index
**now every items time series can be separated by the index**

In [4]:
data = pd.read_csv('/content/drive/MyDrive/M5/ultimative_set.csv')
data = data.drop(columns=['Unnamed: 0'])
data = data.sort_values(by=['item_id', 'date'])
data = data.set_index(['item_id', 'date'])

## split in feature and target matrix


---

In [5]:
X = data.drop(columns=['demand'])
Y = data['demand']

## train_test_split


---

In [6]:
X_train = X.loc[(slice(None), slice(None, "2015")), :]
X_test = X.loc[(slice(None), slice("2015", '2017')), :]

Y_train = Y.loc[(slice(None), slice(None, "2015"))]
Y_test = Y.loc[(slice(None), slice("2015", "2017"))]

## new order after the split


---



*   for the product pooling the whole data must be be sorted by date independent from the 'item_id'
*   therefore sort values by 'date' in the first instance and by 'item_id' in the second 
*   these two instances create the mutli-index of the target matrix (Y)
*   as all items are in one set now the 'item_id' becomes a feature in the feature matrix (X). The only index is the date here.

In [7]:
X_train = X_train.reset_index().sort_values(by=['date', 'item_id']).set_index(['date'])
X_test = X_test.reset_index().sort_values(by=['date', 'item_id']).set_index(['date'])

Y_train = Y_train.reset_index().sort_values(by=['date', 'item_id']).set_index(['date', 'item_id'])
Y_test = Y_test.reset_index().sort_values(by=['date', 'item_id']).set_index(['date', 'item_id'])

## defining the different cost paramerters for special service level and product range



---



*   cu is the mean of the cumulated 'sell_price' column of the training data
*   appending to the corresponding service level co is calculated with (cu - α * cu) / α
*   all the costs are rounded to 2 digits

In [9]:
prods = list(range(0,25))

cu = round(X_train['sell_price'].mean(),2)

co_90 = round(((cu-(0.90*cu))/0.90),2)

co_75 = round(((cu-(0.75*cu))/0.75),2)

co_50 = round(((cu-(0.50*cu))/0.50),2)

## determining some parameters


---


*   the potential range of the params for randomized search are defined
*   the estimator, the scorer and finally the whole param_random is defined

In [ ]:
kernel_bandwidth = np.arange(25, 81)
param_random = dict(kernel_bandwidth = kernel_bandwidth)

scorer_avc = make_scorer(average_costs, greater_is_better=False)

# gwn for 90% service level


---






## randomized search




*   randomized serach is done once for the entire data-set with the corresponding service level costs
*   this step was done with the server of the University of Würzburg because of the high resource consumption 
*   the results for the best params are: kernel_bandwidth = 30

In [ ]:
gwn = GaussianWeightedNewsvendor(cu = cu, co = co_90)
random = RandomizedSearchCV(gwn, param_random, cv=5, scoring = scorer_avc, n_jobs=8)
random.fit(X_train, Y_train)
print(random.best_params_)

## newsvendor-model 90%



---



*   the model is trained and tested once for entire data-set with the best hyperparameters
*   the prediction is the output

In [10]:
gwn = GaussianWeightedNewsvendor(cu = cu, co = co_90, kernel_bandwidth = 30)

gwn.fit(X_train, Y_train)
preds_90 = gwn.predict(X_test)

## reorder the data




---



*   create a pandas frame 'comp_90' where the real values, the predictions and the predictions from the saa model are in columns next to each other

In [11]:
pred_90 = pd.DataFrame(preds_90, columns=['pred_gwn_90'])
comp_90 = Y.loc[(slice(None), slice("2015", "2017"))].reset_index().sort_values(by=['date', 'item_id']).set_index('date')
comp_90 = comp_90.reset_index()
comp_90['pred_gwn_90'] = pred_90[['pred_gwn_90']]
comp_90 = comp_90.sort_values(by=['item_id', 'date']).set_index(['item_id', 'date'])

saa_90 = pd.read_csv('/content/drive/MyDrive/M5/SAA_pred_90.csv').set_index(['item_id', 'date'])

comp_90 = pd.concat([comp_90, saa_90], axis = 1)

## average costs and prescriptivess score


---



*   create a loop where the average_costs (the prescriptivess score) of each item are calculated

*   save the results into a pandas frame

In [12]:
result_gwn_90_avc = []

for prod in prods:
  avc = round(average_costs(comp_90['demand'].loc[prod], comp_90['pred_gwn_90'].loc[prod], cu = cu, co = co_90),2)
  result_gwn_90_avc.append(avc)

In [13]:
GWN_pooled_avc_90 = pd.DataFrame(result_gwn_90_avc, columns=['GWN_avc_90'])
GWN_pooled_avc_90.index.name = 'item_id'

In [15]:
result_gwn_90_pscr = []

for prod in prods:
  pscr = round(prescriptiveness_score(comp_90['demand'].loc[prod], comp_90['pred_gwn_90'].loc[prod],
                                      comp_90['SAA_pred_90'].loc[prod], cu = cu, co = co_90),2)
  result_gwn_90_pscr.append(pscr)

In [19]:
GWN_pooled_pscr_90 = pd.DataFrame(result_gwn_90_pscr, columns=['GWN_pscr_90'])
GWN_pooled_pscr_90.index.name = 'item_id'

# gwn for 75% service level


---






## randomized search




*   randomized serach is done once for the entire data-set with the corresponding service level costs
*   this step was done with the server of the University of Würzburg because of the high resource consumption 
*   the results for the best params are: kernel_bandwidth = 28

In [ ]:
gwn = GaussianWeightedNewsvendor(cu = cu, co = co_75)
random = RandomizedSearchCV(gwn, param_random, cv=5, scoring = scorer_avc, n_jobs=8)
random.fit(X_train, Y_train)
print(random.best_params_)

## newsvendor-model 75%



---



*   the model is trained and tested once for entire data-set with the best hyperparameters
*   the prediction is the output

In [18]:
gwn = GaussianWeightedNewsvendor(cu = cu, co = co_75, kernel_bandwidth = 28)

gwn.fit(X_train, Y_train)
preds_75 = gwn.predict(X_test)

## reorder the data




---



*   create a pandas frame 'comp_75' where the real values, the predictions and the predictions from the saa model are in columns next to each other

In [20]:
pred_75 = pd.DataFrame(preds_75, columns=['pred_gwn_75'])
comp_75 = Y.loc[(slice(None), slice("2015", "2017"))].reset_index().sort_values(by=['date', 'item_id']).set_index('date')
comp_75 = comp_75.reset_index()
comp_75['pred_gwn_75'] = pred_75[['pred_gwn_75']]
comp_75 = comp_75.set_index(['item_id', 'date'])

saa_75 = pd.read_csv('/content/drive/MyDrive/M5/SAA_pred_75.csv').set_index(['item_id', 'date'])

comp_75 = pd.concat([comp_75, saa_75], axis = 1)

## average costs and prescriptivess score


---



*   create a loop where the average_costs (the prescriptivess score) of each item are calculated

*   save the results into a pandas frame

In [21]:
result_gwn_75_avc = []

for prod in prods:
  avc = round(average_costs(comp_75['demand'].loc[prod], comp_75['pred_gwn_75'].loc[prod], cu = cu, co = co_75),2)
  result_gwn_75_avc.append(avc)

In [22]:
GWN_pooled_avc_75 = pd.DataFrame(result_gwn_75_avc, columns=['GWN_avc_75'])
GWN_pooled_avc_75.index.name = 'item_id'

In [24]:
result_gwn_75_pscr = []

for prod in prods:
  pscr = round(prescriptiveness_score(comp_75['demand'].loc[prod], comp_75['pred_gwn_75'].loc[prod],
                                      comp_75['SAA_pred_75'].loc[prod], cu = cu, co = co_75),2)
  result_gwn_75_pscr.append(pscr)

In [25]:
GWN_pooled_pscr_75 = pd.DataFrame(result_gwn_75_pscr, columns=['GWN_pscr_75'])
GWN_pooled_pscr_75.index.name = 'item_id'

# gwn for 50% service level


---






## randomized search




*   randomized serach is done once for the entire data-set with the corresponding service level costs
*   this step was done with the server of the University of Würzburg because of the high resource consumption 
*   the results for the best params are: kernel_bandwidth = 31

In [ ]:
gwn = GaussianWeightedNewsvendor(cu = cu, co = co_50)
random = RandomizedSearchCV(gwn, param_random, cv=5, scoring = scorer_avc, n_jobs=8)
random.fit(X_train, Y_train)
print(random.best_params_)

## newsvendor-model 50%



---



*   the model is trained and tested once for entire data-set with the best hyperparameters
*   the prediction is the output

In [26]:
gwn = GaussianWeightedNewsvendor(cu = cu, co = co_50, kernel_bandwidth = 31)

gwn.fit(X_train, Y_train)
preds_50 = gwn.predict(X_test)

## reorder the data




---



*   create a pandas frame 'comp_50' where the real values, the predictions and the predictions from the saa model are in columns next to each other

In [27]:
pred_50 = pd.DataFrame(preds_50, columns=['pred_gwn_50'])
comp_50 = Y.loc[(slice(None), slice("2015", "2017"))].reset_index().sort_values(by=['date', 'item_id']).set_index('date')
comp_50 = comp_50.reset_index()
comp_50['pred_gwn_50'] = pred_50[['pred_gwn_50']]
comp_50 = comp_50.set_index(['item_id', 'date'])

saa_50 = pd.read_csv('/content/drive/MyDrive/M5/SAA_pred_50.csv').set_index(['item_id', 'date'])

comp_50 = pd.concat([comp_50, saa_50], axis = 1)

## average costs and prescriptivess score


---



*   create a loop where the average_costs (the prescriptivess score) of each item are calculated

*   save the results into a pandas frame

In [29]:
result_gwn_50_avc = []

for prod in prods:
  avc = round(average_costs(comp_50['demand'].loc[prod], comp_50['pred_gwn_50'].loc[prod], cu = cu, co = co_50),2)
  result_gwn_50_avc.append(avc)

In [30]:
GWN_pooled_avc_50 = pd.DataFrame(result_gwn_50_avc, columns=['GWN_avc_50'])
GWN_pooled_avc_50.index.name = 'item_id'

In [31]:
result_gwn_50_pscr = []

for prod in prods:
  pscr = round(prescriptiveness_score(comp_50['demand'].loc[prod], comp_50['pred_gwn_50'].loc[prod],
                                      comp_50['SAA_pred_50'].loc[prod], cu = cu, co = co_50),2)
  result_gwn_50_pscr.append(pscr)

In [32]:
GWN_pooled_pscr_50 = pd.DataFrame(result_gwn_50_pscr, columns=['GWN_pscr_50'])
GWN_pooled_pscr_50.index.name = 'item_id'

# merge the pandas frames


---

In [33]:
GWN_pooled_avc = pd.concat([GWN_pooled_avc_90, GWN_pooled_avc_75, GWN_pooled_avc_50], axis=1)
GWN_pooled_avc

,GWN_avc_90,GWN_avc_75,GWN_avc_50
item_id,,,
0,2.70,5.48,9.41
1,1.37,2.58,4.41
2,2.46,4.78,8.16
3,1.99,4.33,7.97
4,2.32,5.16,9.69
5,1.53,2.96,5.31
6,1.78,3.79,7.17
7,1.70,3.59,6.97
8,1.49,2.90,5.21


In [34]:
GWN_pooled_pscr = pd.concat([GWN_pooled_pscr_90, GWN_pooled_pscr_75, GWN_pooled_pscr_50], axis=1)
GWN_pooled_pscr

,GWN_pscr_90,GWN_pscr_75,GWN_pscr_50
item_id,,,
0,0.28,0.27,0.25
1,0.24,0.35,0.30
2,0.16,0.15,0.12
3,0.62,0.57,0.48
4,0.28,0.17,0.13
5,0.27,0.30,0.17
6,0.47,0.47,0.40
7,0.38,0.25,0.13
8,0.23,0.22,0.19


# save the results


---

In [35]:
GWN_pooled_avc.to_csv('/content/drive/MyDrive/M5/GWN_pooled_avc.csv')

GWN_pooled_pscr.to_csv('/content/drive/MyDrive/M5/GWN_pooled_pscr.csv')